In [44]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [45]:
df = pd.read_csv('../elections_33.csv', encoding='utf-8')

In [46]:
df.head()

,Unnamed: 0,Code du département,Libellé du département,Code de la commune,Libellé de la commune,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,...,% Voix/Exp,Candidat_ID,Année,% Blancs/Ins,% Blancs/Vot,Nuls,% Nuls/Ins,% Nuls/Vot,Blancs,N°Panneau
0,49596,33,GIRONDE,1,ABZAC,1091,195,17.87,896,82.13,...,16.63,1,1995,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,49597,33,GIRONDE,2,AILLAS,530,92,17.36,438,82.64,...,8.49,1,1995,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,49598,33,GIRONDE,3,AMBARES & LAGRAVE,6930,1306,18.85,5624,81.15,...,16.40,1,1995,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,49599,33,GIRONDE,4,AMBES,1834,295,16.09,1539,83.91,...,11.89,1,1995,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,49600,33,GIRONDE,5,ANDERNOS LES BAINS,6551,1422,21.71,5129,78.29,...,13.44,1,1995,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Création d'un nouveau dataframe propre

In [47]:
# create new dataframe with only the columns we need
cols = [
    'Annee', 'Code de la commune', 'Libellé de la commune', 'Inscrits', 'Abstentions', 'Votants', 'Exprimés',
    'Blancs_et_nuls',
    'Sexe', 'Nom Complet', 'Voix', 'Candidat_ID',
]

In [48]:
df_final = pd.DataFrame(columns=cols)

In [49]:
df_final['Annee'] = df['Année']
df_final['Code de la commune'] = df['Code de la commune']
df_final['Libellé de la commune'] = df['Libellé de la commune']
df_final['Inscrits'] = df['Inscrits']
df_final['Abstentions'] = df['Abstentions']
df_final['Votants'] = df['Votants']
df_final['Exprimés'] = df['Exprimés']

# Blancs_et_nuls we need to take the 'Blancs_et_nuls' column and add 'Blancs' and 'Nuls' columns on top of it
df['Blancs'] = df['Blancs'].fillna(0)
df['Nuls'] = df['Nuls'].fillna(0)
df_final['Blancs_et_nuls'] = df['Blancs_et_nuls'] + df['Blancs'] + df['Nuls']

df_final['Sexe'] = df['Sexe'].fillna('N/A')
df_final['Nom Complet'] = df['Nom'] + ' ' + df["Prénom"]
df_final['Voix'] = df['Voix'].fillna(0)
df_final['Candidat_ID'] = df['Candidat_ID'].fillna(0)

In [50]:
df_final['Nom Complet'].unique()

array(['LE PEN J.MARIE', 'CHIRAC JACQUES', 'JOSPIN LIONEL',
       'VOYNET DOMINIQUE', 'LEPAGE CORINNE', 'LE PEN JEAN-MARIE',
       'BOUTIN CHRISTINE', 'MADELIN ALAIN', 'LAGUILLER ARLETTE',
       'BESANCENOT Olivier', 'BUFFET Marie-George', 'SCHIVARDI Gérard',
       'BAYROU François', 'BOVÉ José', 'de VILLIERS Philippe',
       'ROYAL Ségolène', 'NIHOUS Frédéric', 'LAGUILLER Arlette',
       'SARKOZY Nicolas', 'CHEMINADE Jacques', 'LE PEN Marine',
       'MACRON Emmanuel', 'MÉLENCHON Jean-Luc', 'FILLON François',
       'HAMON Benoît', 'LASSALLE Jean', 'DUPONT-AIGNAN Nicolas',
       'POUTOU Philippe', 'ASSELINEAU François', 'ARTHAUD Nathalie',
       'HIDALGO Anne'], dtype=object)

In [51]:
political_lean = {
    'LE PEN J.MARIE': 1.0,  # Far-right
    'CHIRAC JACQUES': 0.4,  # Center-right
    'JOSPIN LIONEL': -0.4,  # Center-left
    'VOYNET DOMINIQUE': -0.8,  # Green left
    'LEPAGE CORINNE': -0.6,  # Ecologist
    'LE PEN JEAN-MARIE': 1.0,  # Far-right
    'BOUTIN CHRISTINE': 0.6,  # Christian democracy
    'MADELIN ALAIN': 0.6,  # Liberal conservatism
    'LAGUILLER ARLETTE': -1.0,  # Far-left
    'BESANCENOT Olivier': -1.0,  # Far-left
    'BUFFET Marie-George': -0.6,  # Left-wing
    'SCHIVARDI Gérard': 0.0,  # Centrist
    'BAYROU François': 0.2,  # Center to center-right
    'BOVÉ José': -0.6,  # Ecologist
    'de VILLIERS Philippe': 0.8,  # Right-wing
    'ROYAL Ségolène': -0.4,  # Center-left
    'NIHOUS Frédéric': 0.0,  # Centrist
    'SARKOZY Nicolas': 0.6,  # Center-right
    'CHEMINADE Jacques': -0.2,  # Slightly left
    'LE PEN Marine': 1.0,  # Far-right
    'MACRON Emmanuel': 0.2,  # Center to center-right
    'MÉLENCHON Jean-Luc': -0.8,  # Far-left
    'FILLON François': 0.6,  # Center-right
    'HAMON Benoît': -0.4,  # Center-left
    'LASSALLE Jean': 0.0,  # Centrist
    'DUPONT-AIGNAN Nicolas': 0.4,  # Center-right
    'POUTOU Philippe': -1.0,  # Far-left
    'ASSELINEAU François': 0.0,  # Centrist
    'ARTHAUD Nathalie': -1.0,  # Far-left
    'HIDALGO Anne': -0.4,  # Center-left
}


df_final['Orientation'] = df_final['Nom Complet'].map(political_lean)

In [52]:
df_final.head()

,Annee,Code de la commune,Libellé de la commune,Inscrits,Abstentions,Votants,Exprimés,Blancs_et_nuls,Sexe,Nom Complet,Voix,Candidat_ID,Orientation
0,1995,1,ABZAC,1091,195,896,860,36.0,M,LE PEN J.MARIE,143,1,1.0
1,1995,2,AILLAS,530,92,438,424,14.0,M,LE PEN J.MARIE,36,1,1.0
2,1995,3,AMBARES & LAGRAVE,6930,1306,5624,5468,156.0,M,LE PEN J.MARIE,897,1,1.0
3,1995,4,AMBES,1834,295,1539,1506,33.0,M,LE PEN J.MARIE,179,1,1.0
4,1995,5,ANDERNOS LES BAINS,6551,1422,5129,4994,135.0,M,LE PEN J.MARIE,671,1,1.0


## Calcul des pourcentages

In [53]:
df_final['Blancs_et_nuls'] = df_final['Blancs_et_nuls'].fillna(0)

In [54]:
df_final['Pourcentage_Blancs_et_nuls'] = df_final['Blancs_et_nuls'] / df_final['Exprimés'] * 100
df_final['Pourcentage_Abstentions'] = df_final['Abstentions'] / df_final['Inscrits'] * 100
df_final['Pourcentage_Votants'] = df_final['Votants'] / df_final['Inscrits'] * 100

In [55]:
# export to csv
df_final.head()

,Annee,Code de la commune,Libellé de la commune,Inscrits,Abstentions,Votants,Exprimés,Blancs_et_nuls,Sexe,Nom Complet,Voix,Candidat_ID,Orientation,Pourcentage_Blancs_et_nuls,Pourcentage_Abstentions,Pourcentage_Votants
0,1995,1,ABZAC,1091,195,896,860,36.0,M,LE PEN J.MARIE,143,1,1.0,4.186047,17.873511,82.126489
1,1995,2,AILLAS,530,92,438,424,14.0,M,LE PEN J.MARIE,36,1,1.0,3.301887,17.358491,82.641509
2,1995,3,AMBARES & LAGRAVE,6930,1306,5624,5468,156.0,M,LE PEN J.MARIE,897,1,1.0,2.852963,18.845599,81.154401
3,1995,4,AMBES,1834,295,1539,1506,33.0,M,LE PEN J.MARIE,179,1,1.0,2.191235,16.085060,83.914940
4,1995,5,ANDERNOS LES BAINS,6551,1422,5129,4994,135.0,M,LE PEN J.MARIE,671,1,1.0,2.703244,21.706610,78.293390


In [56]:
df_final.to_csv('elections_33_clean.csv', index=False)
for col in df_final.columns:
    print(col, df_final[col].dtype)

Annee int64
Code de la commune int64
Libellé de la commune object
Inscrits int64
Abstentions int64
Votants int64
Exprimés int64
Blancs_et_nuls float64
Sexe object
Nom Complet object
Voix int64
Candidat_ID int64
Orientation float64
Pourcentage_Blancs_et_nuls float64
Pourcentage_Abstentions float64
Pourcentage_Votants float64


# Ajout inflation

In [57]:
df_inflation = pd.read_csv('./clean_inflation.csv', encoding='utf-8')

In [58]:
df_inflation.head()

,Période,Indice des prix
0,2024-03,2.3
1,2024-02,3.6
2,2024-01,4.5
3,2023-12,5.9
4,2023-11,7.0


In [59]:
# Convert the "Période" column to datetime format
df_inflation['Période'] = pd.to_datetime(df_inflation['Période'], format='%Y-%m')

# Extract the year from the "Période" column
df_inflation['Year'] = df_inflation['Période'].dt.year

# Group the data by 'Year' and calculate the mean of "Indice des prix"
mean_indice_by_year = df_inflation.groupby('Year')['Indice des prix'].mean().reset_index()


In [60]:
df_inflation["Year"].unique()

array([2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014,
       2013, 2012], dtype=int32)

In [61]:
# Fill the 'Inflation' column with the mean inflation of the corresponding year
df_final['Inflation'] = None

# If not available, do nothing
for year in df_inflation["Year"].unique():
    inflation = mean_indice_by_year[mean_indice_by_year['Year'] == year]['Indice des prix'].values[0]
    df_final.loc[df_final['Annee'] == year, 'Inflation'] = inflation